<a href="https://www.kaggle.com/code/abdulrahmanalabrash/ev-station-optimization-with-or-tools?scriptVersionId=180399633" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

 # ChargeWise Navigator: A Strategic Model for Optimizing Electric Vehicle Refueling Locations with OR Tools






## Import required libraries

In [1]:
from ortools.linear_solver import pywraplp
from itertools import product
import pandas as pd
import folium
from itertools import combinations
solver = pywraplp.Solver.CreateSolver('SCIP')

In [2]:
# Read the CSV file with latitude and longitude data

data_izmir = pd.read_csv('/kaggle/input/optimization-using-or/Copy of zmir_Stations - Sheet1.csv')

toll_data= pd.read_csv('/kaggle/input/optimization-using-or/Name of the tolls.csv')


# Extract latitude and longitude columns from the DataFrame
latitudes = data_izmir['Latitude']
longitudes = data_izmir['Longitude']

Lat=toll_data['lat']
Lon=toll_data['lon']

# Create a list of existing gas station locations
existing_stations = list(zip(latitudes, longitudes))
toll_data

,Name,lat,lon
0,Şehitlik Gişeleri,26.993470,38.397191
1,Seferihisar Gişeleri,26.871305,38.365339
2,Urla Gişesi,26.781988,38.322773
3,Karaburun Gişeleri,26.686092,38.303068
4,Zeytinler Gişesi,26.564120,38.285035
5,Alaçatı Gişeleri,26.382368,38.280521
6,Çeşme Gişeleri,26.310192,38.295976
7,Işıkkent Gişeler Taksi,27.233711,38.351512
8,Tahtalıçay Gişeleri,27.217107,38.272651
9,Torbalı Gişeleri,27.338020,38.194293


## Defining lists and parameters

In [3]:
# Set of Potential Stations 
K = ["ÇEŞME", "URLA", "SEFERIHISAR", "GAZIEMIR", "TAHTALIÇAY", "OTOYOLAYR",  "KARŞIYAKA", "BELEVI", "GERMENCIK", "ŞEVKETIYE"]

# Set of all OD pairs
Q = [
    "ÇEŞME-ALAÇATI-ZEYTINLER", "ÇEŞME-ALAÇATI-ZEYTINLER-KARABURUN", 
    "ÇEŞME-ALAÇATI-ZETINLER-KARABURUN-URLA", "ALAÇATI-ZEYTNILER-KARABURUN", 
    "ALAÇATI-ZEYTNILER-KARABURUN-URLA", "ALAÇATI-ZEYTNILER-KARABURUN-URLA-SEFERIHISAR", 
    "ZEYTNILER-KARABURUN-URLA", "ZEYTNILER-KARABURUN-URLA-SEFERIHISAR", 
    "ZEYTNILER-KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK", "KARABURUN-URLA-SEFERIHISAR", 
    "KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK", "KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA", 
    "URLA-SEFERIHISAR-ŞEHITLIK", "URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA", 
    "URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR", "SEFERIHISAR-ŞEHITLIK-BALÇOVA", 
    "SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR", "SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR-OTOYOLAYR", 
    "ŞEHITLIK-BALÇOVA-GAZIEMIR", "ŞEHITLIK-BALÇOVA-GAZIEMIR-OTOYOLAYR", 
    "ŞEHITLIK-BALÇOVA-GAZIEMIR-OTOYOLAYR-IŞIKKENT", "BALÇOVA-GAZIEMIR-OTOYOLAYR", 
    "BALÇOVA-GAZIEMIR-OTOYOLAYR-IŞIKKENT", "BALÇOVA-GAZIEMIR-OTOYOLAYR-IŞIKKENT-TAHTALIÇAY", 
    "ŞEVKETIYE-GERMENCIK-BELEVI", "ŞEVKETIYE-GERMENCIK-BELEVI-TORBALI", 
    "ŞEVKETIYE-GERMENCIK-BELEVI-TORBALI-TAHTALIÇAY"
]

# Flow volume on the OD pair
f = [
    8235, 9173, 9926, 11606, 12285, 18276, 12520, 22538, 19950, 
    22917, 31085, 32599, 33345, 37703, 41779, 45495, 48642, 53071, 
    50777, 55778, 55715, 59881, 58808, 55010, 24441, 27419, 30014
]

# The number of stations to be opened
P = 4

# Initialize the coverage dictionary
C = {}

# Fill the coverage dictionary
for k in K:
    for q in Q:
        if k in q:
            C[(k, q)] = 1
        else:
            C[(k, q)] = 0

C


{('ÇEŞME', 'ÇEŞME-ALAÇATI-ZEYTINLER'): 1,
 ('ÇEŞME', 'ÇEŞME-ALAÇATI-ZEYTINLER-KARABURUN'): 1,
 ('ÇEŞME', 'ÇEŞME-ALAÇATI-ZETINLER-KARABURUN-URLA'): 1,
 ('ÇEŞME', 'ALAÇATI-ZEYTNILER-KARABURUN'): 0,
 ('ÇEŞME', 'ALAÇATI-ZEYTNILER-KARABURUN-URLA'): 0,
 ('ÇEŞME', 'ALAÇATI-ZEYTNILER-KARABURUN-URLA-SEFERIHISAR'): 0,
 ('ÇEŞME', 'ZEYTNILER-KARABURUN-URLA'): 0,
 ('ÇEŞME', 'ZEYTNILER-KARABURUN-URLA-SEFERIHISAR'): 0,
 ('ÇEŞME', 'ZEYTNILER-KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK'): 0,
 ('ÇEŞME', 'KARABURUN-URLA-SEFERIHISAR'): 0,
 ('ÇEŞME', 'KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK'): 0,
 ('ÇEŞME', 'KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA'): 0,
 ('ÇEŞME', 'URLA-SEFERIHISAR-ŞEHITLIK'): 0,
 ('ÇEŞME', 'URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA'): 0,
 ('ÇEŞME', 'URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR'): 0,
 ('ÇEŞME', 'SEFERIHISAR-ŞEHITLIK-BALÇOVA'): 0,
 ('ÇEŞME', 'SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR'): 0,
 ('ÇEŞME', 'SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR-OTOYOLAYR'): 0,
 ('ÇEŞME', 'ŞEHITLIK-BALÇOVA-GAZIEMIR'): 

## Defining variables

In [4]:
# y = 1 If fq is captured otherwise 0
y = {}
for q in Q:
    y[q] = solver.BoolVar(f'y_{q}')
# x = 1 if a station is located at k otherwise 0
x = {}
for k in K:
    x[k] = solver.BoolVar(f'x_{k}')

## Visualisation 

### In Order to get a better idea about the data that we have, we want to create a map that helps us achieving this goal

### Showing existing gas stations in Izmir region

In [5]:
# Create a map centered around Izmir
izmir_map = folium.Map(location=[38.4237, 27.1428], zoom_start=9)  # You can adjust the coordinates and zoom level as needed

# Add markers for gas stations
for i in range(len(latitudes)):
    lat = latitudes[i]
    lon = longitudes[i]
    folium.CircleMarker([lat, lon],  color="red", radius=1, weight=2).add_to(izmir_map)  

    
izmir_map



### Showing toll roads in Izmir region

In [6]:
point_coordinates = {
    "Şehitlik Gişeleri": (38.397191, 26.99347),
    "Seferihisar Gişeleri": (38.365339, 26.871305),
    "Urla Gişesi": (38.322773, 26.781988),
    "Karaburun Gişeleri": (38.30306799999999, 26.686092),
    "Zeytinler Gişesi": (38.285035, 26.5641199),
    "Alaçatı Gişeleri": (38.280521, 26.3823682),
    "Çeşme Gişeleri": (38.295976, 26.3101921),
    "Işıkkent Gişeler Taksi": (38.3515118, 27.2337111),
    "Tahtalıçay Gişeleri": (38.272651, 27.217107),
    "Torbalı Gişeleri": (38.19429269999999, 27.3380205),
    "Belevi Tolls": (38.0283545, 27.4494247),
    "Germencik Gişesi": (37.878091, 27.5780948),
    "ŞEVKETİYE": (37.8398194, 27.838241),
    "Balçova": (38.3846169, 27.0581702),
    "Gaziemir": (38.32515, 27.12252),
    "OTOYOL AYR": (38.4402007, 27.2261902),
    "ÜNİVERSİTE AYR": (38.4811447, 27.1582345),
    "KARŞIYAKA": (38.5488434, 27.0366772),
    "End": (37.847296, 27.9168236),
}

# Create a map centered around a specific point (e.g., the first point)
map_center = list(point_coordinates.values())[0]




In [7]:
# Add points to the map
for name, coordinates in point_coordinates.items():
    folium.Marker(location=coordinates, popup=name).add_to(izmir_map)

izmir_map

### Showing the number of cars passing through each toll road and coloring it to have a better understanding of the traffic movement

In [8]:
# Define a color mapping function based on car volume
def color_mapping(car_volume):
    if car_volume >= 60:  # Red for busy
        return 'red'
    elif car_volume >= 30:  # Yellow for normal
        return 'yellow'
    else:  # Green for empty
        return 'green'

# Locations and car volumes
locations = [
    (38.2695061, 26.3335397, "ALAÇATI - ÇEŞME", 4.889),
    (38.2699406, 26.4837731, "ZEYTİNLER - ALAÇATI", 11.581),
    (38.2925787, 26.628312, "KARABURUN - ZEYTİNLER", 11.05),
    (38.2995843, 26.7371453, "URLA - KARABURUN", 12.187),
    (38.3416037, 26.8233193, "SEFERİHİSAR - URLA", 14.325),
    (38.3761042, 26.930436, "ŞEHİTLİK - SEFERİHİSAR", 42.24),
    (38.3565197, 27.088229, "BALÇOVA - GAZİEMİR", 58.084),
    (38.3651343, 27.1766346, "GAZİEMİR - OTOYOL AYR.", 70.784),
    (38.4451636, 27.2107121, "OTOYOL AYR. - ÜNİVERSİTE AYR.", 101.826),
    (38.4889367, 27.0874592, "ÜNİVERSİTE AYR. - KARŞIYAKA", 91.069),
    (38.3278117, 27.2364814, "IŞIKKENT - TAHTALIÇAY", 40.396),
    (38.2226299, 27.2518079, "TAHTALIÇAY - TORBALI", 36.351),
    (38.0979131, 27.3747175, "TORBALI - BELEVİ", 26.882),
    (37.9621568, 27.5120466, "BELEVİ - GERMENCİK", 25.128),
    (37.8703384, 27.736923, "GERMENCİK - ŞEVKETİYE", 21.315)
]

# Add markers with car volumes as labels and color-coded based on the mapping function
for location in locations:
    car_volume = location[3]
    color = color_mapping(car_volume)
    folium.Marker(
        location=[location[0], location[1]],
        popup=f"{location[2]}\nNumber of cars: {location[3]}",
        icon=folium.Icon(color=color)
    ).add_to(izmir_map)
izmir_map

/tmp/ipykernel_19/3889126832.py:36: UserWarning: color argument of Icon should be one of: {'green', 'darkpurple', 'darkgreen', 'purple', 'lightgray', 'gray', 'lightblue', 'cadetblue', 'beige', 'white', 'lightgreen', 'darkred', 'red', 'darkblue', 'black', 'pink', 'lightred', 'orange', 'blue'}.
  icon=folium.Icon(color=color)


## Defining constrains

In [9]:
#prevents the capturing of path q, unless a suitable combination of stations is selected.
for q in Q:
    solver.Add(solver.Sum(C[(k, q)] * x[k] for k in K) >= y[q])

#ensures that the number of opened stations equals to p
solver.Add(solver.Sum(x[k] for k in K) == P)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7c0039bbf450> >

## Defining objective function and solving the problem

### The objective is to maximize the total captured flow volume.



In [10]:
solver.Maximize(solver.Sum(f[i] * y[q] for i, q in enumerate(Q)))
status = solver.Solve()

#This shows if there's an optimal solution and which OD pairs are covered in the optimal solution.
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for i in Q:
        print(y[i].name(), ' = ', y[i].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 859974.0
y_ÇEŞME-ALAÇATI-ZEYTINLER  =  0.0
y_ÇEŞME-ALAÇATI-ZEYTINLER-KARABURUN  =  0.0
y_ÇEŞME-ALAÇATI-ZETINLER-KARABURUN-URLA  =  1.0
y_ALAÇATI-ZEYTNILER-KARABURUN  =  0.0
y_ALAÇATI-ZEYTNILER-KARABURUN-URLA  =  1.0
y_ALAÇATI-ZEYTNILER-KARABURUN-URLA-SEFERIHISAR  =  1.0
y_ZEYTNILER-KARABURUN-URLA  =  1.0
y_ZEYTNILER-KARABURUN-URLA-SEFERIHISAR  =  1.0
y_ZEYTNILER-KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK  =  1.0
y_KARABURUN-URLA-SEFERIHISAR  =  1.0
y_KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK  =  1.0
y_KARABURUN-URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA  =  1.0
y_URLA-SEFERIHISAR-ŞEHITLIK  =  1.0
y_URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA  =  1.0
y_URLA-SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR  =  1.0
y_SEFERIHISAR-ŞEHITLIK-BALÇOVA  =  1.0
y_SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR  =  1.0
y_SEFERIHISAR-ŞEHITLIK-BALÇOVA-GAZIEMIR-OTOYOLAYR  =  1.0
y_ŞEHITLIK-BALÇOVA-GAZIEMIR  =  1.0
y_ŞEHITLIK-BALÇOVA-GAZIEMIR-OTOYOLAYR  =  1.0
y_ŞEHITLIK-BALÇOVA-GAZIEMIR-OTOYOLAYR-IŞIKKENT  =  1.0
y_BALÇOVA-GAZIEMIR-OT

### The optimized solution for placing the charge station would be at URLA, SEFERIHISAR, GAZIEMIR and ŞEVKETIYE)  

In [11]:
for key, val in x.items():
    print(f"{key}: {val.solution_value()}")

ÇEŞME: 0.0
URLA: 1.0
SEFERIHISAR: 1.0
GAZIEMIR: 1.0
TAHTALIÇAY: 0.0
OTOYOLAYR: 0.0
KARŞIYAKA: 0.0
BELEVI: 0.0
GERMENCIK: 0.0
ŞEVKETIYE: 1.0


### The sum of flow volume in our optimized solution 

In [12]:
sum(f)

888988

### The proportion of the total flow volume that is covered by the selected gas stations in the optimal solution.

In [13]:
solver.Objective().Value()/sum(f)

0.9673628890378723

In [14]:


# Mapping of selected station names to their coordinates
station_coordinates_map = {
    "SEFERIHISAR": "Seferihisar Gişeleri",
    "URLA": "Urla Gişesi",
    "ŞEVKETIYE": "ŞEVKETİYE",
    "GAZIEMIR": "Gaziemir"
}



# Create a map centered around a specific point (e.g., the first point in point_coordinates)
map_center = [38.4237, 27.1428]
new_map = folium.Map(location=map_center, zoom_start=9)

# Add markers for all stations
for station, coordinates in point_coordinates.items():
    color = 'red' if station in station_coordinates_map.values() else 'gray'
    folium.Marker(location=coordinates, popup=station, icon=folium.Icon(color=color)).add_to(new_map)

# Add markers for selected gas stations with a unique color
for station in station_coordinates_map:
    mapped_station_name = station_coordinates_map.get(station)
    if mapped_station_name:
        coordinates = point_coordinates[mapped_station_name]
        folium.Marker(location=coordinates, popup=station, icon=folium.Icon(color='blue')).add_to(new_map)
    else:
        print(f"Coordinates not found for station: {station}")

# Display the map
new_map
